Google Connect

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import library

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
# %matplotlib ipympl

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.metrics import f1_score

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

import seaborn as sns

Read Data

In [4]:
# Google Colab
train_all = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Advance Data Analysis/Assignment 2/Practice/FIT5149_A2_data/train_data_withlabels.csv')
test_all = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Advance Data Analysis/Assignment 2/Practice/FIT5149_A2_data/test_data_nolabels.csv')

In [5]:
# train_all.head()

In [6]:
# test_all.head()

Needed Columns

In [7]:
day_of_week_train = pd.get_dummies(train_all.dayofweek)
X2_train = train_all.drop('dayofweek', 1)
X3_train = pd.concat([X2_train, day_of_week_train], axis=1)

# X3_train.head()

In [8]:
# print(train_all.shape)
# print(X3_train.shape)

In [9]:
day_of_week_test = pd.get_dummies(test_all.dayofweek)
X2_test = test_all.drop('dayofweek', 1)
X3_test = pd.concat([X2_test, day_of_week_test], axis=1)

# X3_test.head()

In [10]:
# print(test_all.shape)
# print(X3_test.shape)

In [11]:
cols = ['load', 'hourofday',	'dif',	'absdif',	'max', 'var', 'entropy',	'nonlinear',	'hurst', 'Fri',	'Mon',	'Sat',	'Sun',	'Thu', 'Tue',	'Wed']
target = ['ac',	'ev',	'oven',	'wash',	'dryer']

In [12]:
X_train_o = X3_train[cols]
Y_train_o = X3_train[target]
X_test_o = X3_test[cols]

In [13]:
# X_train.head()

In [14]:
# Y_train.head()

In [15]:
# X_test

In [16]:
# print(X_train.shape)
# print(X_train.head()
# .shape)

EDA

In [17]:
# # Price chart
# fig, ax = plt.subplots(figsize=(16, 8))
# plt.plot(train_all['Unnamed: 0'], train_all['load'], label='load', color='gold')
# # plt.title('Gold ' + str(np.min(x_train['Date'])) + ' - ' + str(np.max(x_train['Date'])))
# plt.legend(loc='upper left')
# plt.grid()
# plt.show()

In [18]:
# plt.figure(figsize=(12,10))
# cor = X3_train.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()

In [19]:
# # check missing values in train data
# X3_train.isnull().sum()

In [20]:
# for i in target:
#   print(i)
  
#   g = sns.countplot(train_all[i])
#   g.set_xticklabels(['0','1'])
#   plt.show()

#   count_no_sub = len(train_all[train_all[i]==0])
#   count_sub = len(train_all[train_all[i]==1])
#   print("No of 0 :", count_no_sub)
#   print("No of 1 :", count_sub, '\n')

In [21]:
# # class count
# class_count_0, class_count_1 = X3_train['ac'].value_counts()
      
# # Separate class
# class_0 = X3_train[X3_train['ac'] == 0]
# class_1 = X3_train[X3_train['ac'] == 1]
# # print the shape of the class
# print('class 0:', class_0.shape)
# print('class 1:', class_1.shape)

# class_1_over = class_1.sample(class_count_0, replace=True)

# test_over = pd.concat([class_1_over, class_0], axis=0)

# print("total class of 1 and 0:",test_over['ac'].value_counts())# plot the count after under-sampeling
# test_over['ac'].value_counts().plot(kind='bar', title='count (target)')

1) Train Test Split (Random Split 70:30)

In [22]:
x_train_o, x_valid_o, y_train_o, y_valid_o = train_test_split(X_train_o, Y_train_o)

In [23]:
# x_train.head()

In [24]:
# y_valid.head()

In [25]:
# for i in target:
#   print(i)
#   count_no_sub = len(y_train[y_train[i]==0])
#   count_sub = len(y_train[y_train[i]==1])
#   print("No of 0 :", count_no_sub)
#   print("No of 1 :", count_sub, '\n')

In [26]:
# for i in target:
#   print(i)
#   count_no_sub = len(y_valid[y_valid[i]==0])
#   count_sub = len(y_valid[y_valid[i]==1])
#   print("No of 0 :", count_no_sub)
#   print("No of 1 :", count_sub, '\n')

In [27]:
def random_over_sampeling(X3_train, targ):
  # class count
  class_count_0, class_count_1 = X3_train[targ].value_counts()

  # Separate class
  class_0 = X3_train[X3_train[targ] == 0]
  class_1 = X3_train[X3_train[targ] == 1]

  class_1_over = class_1.sample(class_count_0, replace=True)
  test_over = pd.concat([class_1_over, class_0], axis=0) 

  X_train_rs = test_over[cols]
  Y_train_rs = test_over[target] 



  return X_train_rs, Y_train_rs

In [28]:
def model_description_prediction(model_, x_train, y_train, x_valid, y_valid, targ):
  model_.fit(x_train, y_train[targ])
  y_pred = model_.predict(x_valid)
  species = np.array(y_valid[targ])
  predictions = np.array(y_pred)
  #K-fold cross validation
  results = model_selection.cross_val_score(estimator=model_, X=x_train, y=y_train[targ], cv=kfold, scoring='f1')

  print('For Target -->',targ)
  print('Confusion matrix :\n', confusion_matrix(species, predictions))
  print('Classification report : ', classification_report(species, predictions))
  print('F1 score :', round(f1_score(species, predictions), 5))
  print('F1 score on K folds:', round(results.mean(), 5), 'Standard derivation :', round(results.std(), 5))

K fold cross validation

In [29]:
kfold = model_selection.KFold(n_splits=3, random_state=42, shuffle=True)

Models

In [49]:
model_LR = LogisticRegression(solver='liblinear')
model_DT = DecisionTreeClassifier()
model_RF = RandomForestClassifier(n_estimators=100, bootstrap = True , max_features = 'sqrt')
model_GB = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_features=2, max_depth=2, random_state=0)
model_NB = GaussianNB()
model_SGD = SGDClassifier(loss='modified_huber', shuffle=True, random_state=101)
model_KNN = KNeighborsClassifier(n_neighbors=15)
# model_SVC = svm.SVC(C=0.5, random_state=101)

# models_all = [model_LR, model_DT, model_RF, model_GB, model_NB, model_SGD, model_KNN]
models_all = [model_NB]

In [50]:
for model in models_all:
  print('\n', model)

  for targ in target:

    if model in (model_LR, model_DT):
      #Without Over-Sampling
      print('\n=> No-Sampling')
      model_description_prediction(model, x_train_o ,y_train_o, x_valid_o, y_valid_o, targ)
    else:
      #Random Over-Sampeling
      print('\n==> Random-Over-Sampling')
      
      X_train_rs, Y_train_rs = random_over_sampeling(X3_train, targ)
      x_train_rs, x_valid_rs, y_train_rs, y_valid_rs = train_test_split(X_train_rs, Y_train_rs)

      model_description_prediction(model, x_train_rs, y_train_rs, x_valid_rs, y_valid_rs, targ)


 GaussianNB(priors=None, var_smoothing=1e-09)

==> Random-Over-Sampling
For Target --> ac
Confusion matrix :
 [[75958  3144]
 [ 1387 77772]]
Classification report :                precision    recall  f1-score   support

           0       0.98      0.96      0.97     79102
           1       0.96      0.98      0.97     79159

    accuracy                           0.97    158261
   macro avg       0.97      0.97      0.97    158261
weighted avg       0.97      0.97      0.97    158261

F1 score : 0.97169
F1 score on K folds: 0.9719 Standard derivation : 0.00014

==> Random-Over-Sampling
For Target --> ev
Confusion matrix :
 [[83621 20112]
 [20659 83301]]
Classification report :                precision    recall  f1-score   support

           0       0.80      0.81      0.80    103733
           1       0.81      0.80      0.80    103960

    accuracy                           0.80    207693
   macro avg       0.80      0.80      0.80    207693
weighted avg       0.80      0.80    

Random Forest HyperTuning

In [31]:
# from sklearn import model_selection
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# # number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# # number of features at every split
# max_features = ['aut', 'sqrt']

# # max depth
# max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
# max_depth.append(None)

# # create random grid
# random_grid = {
#  'n_estimators': n_estimators,
#  'max_features': max_features,
#  'max_depth': max_depth
#  }

# # Random search of parameters
# rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# # Fit the model

# rfc_random.fit(x_val, small_train['ac'])
# print(rfc_random.best_params_)

In [ ]:
# rf_tuned_model = RandomForestRegressor(n_estimators=200, max_features='sqrt', max_depth=None, random_state=0)

Hyper tune Gradient Boosting

In [ ]:
# lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]


# for learning_rate in lr_list:
#   print("\nLearning rate:", learning_rate)
#   gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
#   for i in target:
#     print("Target:", i)
#     gb_clf.fit(x_val, train[i])
#     y_pred = gb_clf.predict(x_val_test)
#     predictions = np.array(y_pred)
#     test_y = np.array(test[i])
#     print("F1", f1_score(test[i], np.rint(predictions), labels = 'binary')) #np.rint is rounding array to nearest int
#     print(confusion_matrix(test_y, np.rint(predictions)))

Predictions:

In [ ]:
X_test = X3_test[cols]
tuned_df = pd.DataFrame(columns=['ac','ev','oven','wash','dryer'])
for targ in target:
  model = models_all[0]
  X_train_f, Y_train_f = random_over_sampeling(X3_train, targ)
  model.fit(X_train_f, Y_train_f[targ])
  y_pred = model.predict(X_test_o)
  predictions = np.array(y_pred)
  tuned_df[targ]=predictions

tuned_df1 = tuned_df
tuned_df1.insert(0, 'id', range(1, 1+len(tuned_df)))

tuned_df1.to_csv('/content/drive/MyDrive/Colab Notebooks/Advance Data Analysis/Assignment 2/Practice/Kaggle_test_output_data/test_03.csv',index=False)

KeyboardInterrupt: ignored